In [95]:
import bk.compute
import bk.load
import bk.plot

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

import matplotlib
from matplotlib.patches import Rectangle

# import easydict
# import ruamel

import neuroseries as nts

import cv2
import yaml
import pickle

In [9]:
%matplotlib qt

In [10]:
path =  'Y://elevated_plus_maze/'
config_path = 'Y:\elevated_plus_maze\elevated_plus_maze-BM-2020-12-01\config.yaml'

In [38]:
def onclick(event):
    global x,y,pause
    plt.cla()
    plt.imshow(frame)
    try:
        x
    except:
        x = [0,0]
    try:
        y
    except:
        y = [0,0]
    if (str(event.button) == "MouseButton.LEFT") or (event.button == 1):
        x[0], y[0] = event.xdata, event.ydata
    elif (str(event.button) == "MouseButton.RIGHT") or (event.button == 3):
        x[1], y[1] = event.xdata, event.ydata
    elif (str(event.button) == "MouseButton.MIDDLE") or (event.button == 2):
        fig.canvas.mpl_disconnect(cid)
        pause = False
    for i,j in zip(x,y):
        plt.axvline(i)
        plt.axhline(j)
        plt.draw()

In [23]:
sessions = []
for subdirs, dirs, files in os.walk(path):
    for file in files:
        if file.endswith('filtered.h5'): 
            sessions.append(subdirs)
            break
metadata = []
for subdirs, dirs, files in os.walk(path):
    for file in files:
        if file.endswith('meta.pickle'): 
            metadata.append(os.path.join(subdirs,file))
            break

videos = []
for subdirs, dirs, files in os.walk(path):
    for file in files:
        if file.endswith('.mp4') and not file.endswith('labeled.mp4'): videos.append(os.path.join(subdirs,file))
            
crops_x1 = []
crops_x2 = []
crops_y1 = []
crops_y2 = []
for m in metadata:
    with open(m,'rb') as f:
        data = pickle.load(f)
    x1,x2,y1,y2 = data['data']['cropping_parameters']
    crops_x1.append(x1)
    crops_x2.append(x2)
    crops_y1.append(y1)
    crops_y2.append(y2)
    
    
positions_arms = {'open':np.empty([2,2],dtype='object'),'close':np.empty([2,2],dtype='object'),'center':np.empty([1,2],dtype='object')}
positions = ['up', 'down','left', 'right']

colors = {'open':'b','close':'orange','center':'green'}

In [28]:
session = sessions[0]
os.chdir(session)
pos = bk.load.DLC_pos()
v  = 'Y:\elevated_plus_maze\Rat06-20201123\Rat06_201123_152703\Basler_acA1300-200uc__23039139__20201123_152706224.mp4'

In [39]:
j = 0
for p in positions_arms:
    if p == 'center':
        positions_arms[p] = np.array([[(positions_arms['close'][0][0][1], positions_arms['close'][1][0][0]), (positions_arms['open'][0][1][1], positions_arms['open'][1][1][0])]],dtype='object')
    else:
        for i in range(positions_arms[p].shape[0]):
            try:
                del x,y
            except:
                pass

            cap = cv2.VideoCapture(v)
            ret,frame = cap.read()
            frame=frame[crops_y1[0]:crops_y2[0], crops_x1[0]:crops_x2[0]]
            pause = True
            fig,ax = plt.subplots()
            plt.imshow(frame)
            plt.title(p + positions[j])
            cid = fig.canvas.mpl_connect('button_press_event', onclick)

            while pause:
                plt.pause(0.1)

            plt.close('all')
            positions_arms[p][i] = [x,y]
            j += 1


In [74]:
length = len(pos.as_units('s')['snout']['x'])
whereistherat = {'open':np.zeros([1,length],dtype='object'),'close':np.zeros([1,length],dtype='object'),'center':np.zeros([1,length],dtype='object')}
for p in positions_arms:
    for j in range(positions_arms[p].shape[0]):
        (x1,x2), (y1,y2) = positions_arms[p][j]
        for i in range(length):
            if x1<pos.as_units('s')['snout']['x'].iloc[i]<x2 and y1<pos.as_units('s')['snout']['y'].iloc[i]<y2 :
                whereistherat[p][0][i] = 1
                
intervals = {'open':[],'close':[],'center':[]}
for p in positions_arms:
    difference = np.diff(whereistherat[p][0].astype(np.int8))
    start = pos.index[np.where(difference == 1)[0]]
    end = pos.index[np.where(difference == -1)[0]]
    intervals[p] = nts.IntervalSet(start,end, time_units='us')
    
frequencies = {'open':[],'close':[],'center':[]}
for p in frequencies:
    frequencies[p] = np.sum(whereistherat[p][0])/len(whereistherat[p][0])

In [135]:
plt.figure()

for p in frequencies:
    plt.bar(p,frequencies[p], color=colors[p])
plt.title('Pourcent of time spent')
    
plt.figure()
for p in intervals:
    bk.plot.intervals(intervals[p], time_units='s', col =colors[p])
plt.legend(['Open', 'Close', 'Center'])
plt.title('the rat is in the kitchen')

Text(0.5, 1.0, 'the rat is in the kitchen')

# Trash

In [41]:
colors = ['r', 'b', 'green', 'orange', 'yellow']
cap = cv2.VideoCapture(v)
ret,frame = cap.read()
frame_cropped=frame[crops_y1[0]:crops_y2[0], crops_x1[0]:crops_x2[0]]
pause = True
fig,ax = plt.subplots()

plt.cla()
plt.imshow(frame_cropped)
ax = plt.gca()
rectangles = []
j = 0
for p in positions_arms:
    for i in range(positions_arms[p].shape[0]):
        (x1,x2), (y1,y2) = positions_arms[p][i]
        rectangle = Rectangle((x1,y1), x2-x1, y2-y1,linewidth=1,edgecolor=colors[j],facecolor=colors[j],alpha =0.5)
        ax.add_patch(rectangle)
        j += 1
plt.draw()


In [34]:
plt.scatter(pos.as_units('s')['snout']['x'], pos.as_units('s')['snout']['y'], c = viridis(colors),s=10)
plt.show()

In [120]:
colors = {'open':'b','close':'orange','center':'green'}

plt.figure()
s = plt.subplot(2,1,1)
for p in whereistherat:
    plt.plot(whereistherat[p][0])
    plt.legend(p)
plt.title('the rat is in the kitchen')
plt.subplot(2,1,2, sharex = s)
for p in intervals:
    bk.plot.intervals(intervals[p], time_units='s', col =colors[p])

open
close
center


In [33]:
colors = np.linspace(0,1,len(pos.as_units('s')))
viridis = matplotlib.cm.get_cmap('viridis')


In [32]:
cap = cv2.VideoCapture(v)
ret,frame = cap.read()
frame_cropped=frame[crops_y1[0]:crops_y2[0], crops_x1[0]:crops_x2[0]]
pause = True
fig,ax = plt.subplots()
plt.imshow(frame_cropped)